# Download Meteo Data
This notebook demonstrates how to download meteorological data using the `cht_meteo` library.

In [ ]:
import os
from datetime import datetime
import shutil
import tempfile
import cht_meteo as meteo
from pathlib import Path
import numpy as np

## Set up the temporary directory
Create a temporary directory to store the downloaded data. If the directory already exists, remove it and create a new one.

In [ ]:
meteo_path = Path(tempfile.gettempdir()) / "cht_meteo_data"
if meteo_path.exists():
    shutil.rmtree(meteo_path)
meteo_path.mkdir(parents=True)

## Define parameters
Set the time range, longitude range, latitude range, and storm number for the data to be downloaded.

In [ ]:
time_range = (datetime(2024, 9, 26, 0, 0, 0), datetime(2024, 9, 26, 6, 0, 0)) 
lon_range = (-80.0, -70.0)
lat_range = (30.0, 40.0)
storm_number = "10L"

## Create GFS dataset
Create a GFS dataset with the specified parameters.

In [ ]:
gfs_anl = meteo.dataset(name="gfs_anl_0p50_atlantic",
                        source="gfs_analysis_0p50",                      
                        path=os.path.join(meteo_path, "gfs_anl_0p50_atlantic"),
                        lon_range=lon_range,
                        lat_range=lat_range,)

## Download and collect GFS data
Download the GFS data for the specified time range and collect it.

In [ ]:
gfs_anl.download(time_range)
gfs_anl.collect(time_range)

Make a map plot of the barometric pressure for the first time step.

In [ ]:
gfs_anl.ds.barometric_pressure.isel(time=0).plot()

## Create COAMPS-TC dataset
Create a COAMPS-TC dataset with the specified parameters.

In [ ]:
ctc = meteo.dataset(name="coamps_tc_forecast_atlantic",
                    source="coamps_tc_forecast_s3",
                    path=os.path.join(meteo_path, "coamps_tc_forecast_atlantic"),
                    lon_range=lon_range,
                    lat_range=lat_range,)

## Download and collect COAMPS-TC data
Download the COAMPS-TC data for the specified time range and storm number, then collect it.

In [ ]:
ctc.download(time_range, storm_number=storm_number)
ctc.collect(time_range)

## Interpolate COAMPS-TC data
Interpolate the COAMPS-TC data onto a fixed grid.

In [ ]:
ctcf = ctc.cut_out(x_range=lon_range,
                   y_range=lat_range,
                   dx=0.1, dy=0.1,)

Make a map plot of the barometric pressure for the first time step.

In [ ]:
ctcf.ds.barometric_pressure.isel(time=0).plot()

Let's delete a part of the data to test the merge function.

In [ ]:
ctcf.ds = ctcf.ds.where(ctcf.ds["lat"] < 36, np.nan)
ctcf.ds.barometric_pressure.isel(time=0).plot()

## Merge datasets
Fill gaps in the COAMPS-TC data with GFS data.

In [ ]:
ctcf.merge_datasets([gfs_anl])
ctcf.ds.barometric_pressure.isel(time=0).plot()

## Save data
Write the interpolated COAMPS-TC data to NetCDF and Delft3D formats.

In [ ]:
ctcf.to_netcdf(file_name=str(meteo_path / "coamps_tc_forecast_atlantic.nc"))
ctcf.to_delft3d(file_name=str(meteo_path / "coamps_tc_forecast_atlantic"))

In [ ]:
# Get a list of all files in the directory, excluding directories
all_files = [f for f in meteo_path.glob("*") if f.is_file()]

# Print the list of files
for file in all_files:
    print(file)